In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import json
import math
from tqdm import notebook
import time
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
game_list = pd.read_excel('test_to_excel.xlsx')
game_list

,appid,name,short_description,price,categories,genres,recommendations,release_date,developers,metacritic,image
0,1780820,Chill Corner - Piano & Guitar (Music Album),This DLC added 21 new soundtracks with the pri...,110000.0,"['싱글 플레이어', '다운로드 가능한 콘텐츠', 'Steam 도전 과제', 'St...","['캐주얼', '무료', '인디', '시뮬레이션']",NaN,2021년 12월 17일,['Low-Hi Tech'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
1,1780830,Hundred Days - Napa Valley,Explore the napa valley region and its grape v...,950000.0,"['싱글 플레이어', '다운로드 가능한 콘텐츠', 'Steam 도전 과제', '컨트...","['인디', '시뮬레이션', '전략']",NaN,2021년 12월 21일,['Broken Arms Games'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
2,1780840,Loch Ness,Loch Ness is a 1-4 Player Co-Op Horror Hunter....,1450000.0,"['싱글 플레이어', '멀티플레이어', '협동', '온라인 협동', 'Steam 도...","['인디', '앞서 해보기']",NaN,2021년 12월 3일,['JFi Games'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
3,1780850,Cthulhu Dungeon,《不可名状的地牢》是一款以克苏鲁的呼唤跑团为背景的 战棋+DBG+Roguelike游戏，通...,750000.0,['싱글 플레이어'],"['인디', '전략', '앞서 해보기']",NaN,2022년 2월 18일,['SanYeGame'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
4,1780880,Queue Simulator,Queue simulator brings you your favorite part ...,110000.0,"['싱글 플레이어', 'Steam 도전 과제']","['캐주얼', '인디', '대규모 멀티플레이어', 'RPG', '시뮬레이션']",NaN,2021년 12월 15일,['Just Making Games'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
...,...,...,...,...,...,...,...,...,...,...,...
9872,2163690,Dicknosaurus Prologue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9873,2163700,In The Workplace Of Madness - ONE ROOM DEMO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9874,2163710,Aevumblade Chronicles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9875,2163720,Raige Demo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# 상위 5000개의 데이터를 data에 저장
gamedata = game_list.head(5000)
# gamedata.info()

# genres 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('short_description 열의 결측값의 수 : ', gamedata['short_description'].isnull().sum())

short_description 열의 결측값의 수 :  2293


In [4]:
# 결측값을 빈값으로 대체한다.
gamedata['short_description'] = gamedata['short_description'].fillna('') 
print('short_description 열의 결측값의 수 : ', gamedata['short_description'].isnull().sum())

short_description 열의 결측값의 수 :  0


C:\Users\multicampus\AppData\Local\Temp\ipykernel_7844\1945834072.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gamedata['short_description'] = gamedata['short_description'].fillna('')


In [5]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(gamedata['short_description'])
print('TF-IDF 행렬의 크기 : ', tfidf_matrix.shape)

TF-IDF 행렬의 크기 :  (5000, 13086)


In [6]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (5000, 5000)


In [7]:
gamedata

name_to_index = dict(zip(gamedata['appid'], gamedata.index))
# name_to_index

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = name_to_index[1780840]

# idx 출력해본다.
print(idx)

2


In [8]:

def get_recommendation(appid, cosine_sim=cosine_sim):
    # 선택한 게임의 id로 부터 해당 영화의 인덱스를 받아온다.
    idx = name_to_index[appid]
    
    # 해당 게임과 모든 게임의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 게임을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True )
    
    # 가장 유사한 게임 10개를 받아온다
    sim_scores = sim_scores[1:11]
    
    # 가장 유사한 10개 게임의 인덱스를 얻는다.
    game_indices = [idx[0] for idx in sim_scores]
    
    # 가장 유사한 10개의 영화 제목을 리턴한다.
    return gamedata['name'].iloc[game_indices]

get_recommendation(1780880)
    

819     Grim Tales: Crimson Hollow Collector's Edition
4575                                      Gnome Online
4763                                             动物园怪谈
3975                  Fairground Power Polyp Simulator
4605                                       Village RPG
76                              Play Outside Simulator
2935                            Idle Trading Simulator
190                                   Steps From Above
4732             Lights, Camera, Reaction! IRL Edition
2923                               Thats Life of Ralph
Name: name, dtype: object